# Data format example

In [1]:
import pypesto
import amici
import petab

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Manage PETAB model

In [2]:
folder_base = "/home/yannik/benchmark-models/hackathon_contributions_new_data_format/"
model_name = "Zheng_PNAS2012"

manager = petab.Manager.from_folder(folder_base + model_name)

# print(petab.lint.check_measurement_df(manager.measurement_df))
manager.map_par_sim_to_par_opt()


[['inflowp',
  'k00_01',
  'k00_10',
  'k01_00',
  'k01_02',
  'k01_11',
  'k02_01',
  'k02_03',
  'k02_12',
  'k03_02',
  'k03_13',
  'k10_00',
  'k10_11',
  'k10_20',
  'k11_01',
  'k11_10',
  'k11_12',
  'k11_21',
  'k12_02',
  'k12_11',
  'k12_13',
  'k12_22',
  'k13_03',
  'k13_12',
  'k13_23',
  'k20_10',
  'k20_21',
  'k20_30',
  'k21_11',
  'k21_20',
  'k21_22',
  'k21_31',
  'k22_12',
  'k22_21',
  'k22_23',
  'k22_32',
  'k23_13',
  'k23_22',
  'k30_20',
  'k30_31',
  'k31_21',
  'k31_30',
  'k31_32',
  'k32_22',
  'k32_31',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma']]

## Import model to AMICI

In [3]:
importer = pypesto.objective.Importer(manager)
print(importer.output_folder)
model = importer.model


print("Model parameters:", list(model.getParameterIds()), '\n')
print("Model const parameters:", list(model.getFixedParameterIds()), '\n')
print("Model outputs:   ", list(model.getObservableIds()), '\n')
print("Model states:    ", list(model.getStateIds()), '\n')

/home/yannik/pypesto/doc/example/tmp/Zheng_PNAS2012
Model parameters: ['inflowp', 'k00_01', 'k00_10', 'k01_00', 'k01_02', 'k01_11', 'k02_01', 'k02_03', 'k02_12', 'k03_02', 'k03_13', 'k10_00', 'k10_11', 'k10_20', 'k11_01', 'k11_10', 'k11_12', 'k11_21', 'k12_02', 'k12_11', 'k12_13', 'k12_22', 'k13_03', 'k13_12', 'k13_23', 'k20_10', 'k20_21', 'k20_30', 'k21_11', 'k21_20', 'k21_22', 'k21_31', 'k22_12', 'k22_21', 'k22_23', 'k22_32', 'k23_13', 'k23_22', 'k30_20', 'k30_31', 'k31_21', 'k31_30', 'k31_32', 'k32_22', 'k32_31', 'noiseParameter1_K27me0K36me0', 'noiseParameter1_K27me0K36me1', 'noiseParameter1_K27me0K36me2', 'noiseParameter1_K27me0K36me3', 'noiseParameter1_K27me1K36me0', 'noiseParameter1_K27me1K36me1', 'noiseParameter1_K27me1K36me2', 'noiseParameter1_K27me1K36me3', 'noiseParameter1_K27me2K36me0', 'noiseParameter1_K27me2K36me1', 'noiseParameter1_K27me2K36me2', 'noiseParameter1_K27me2K36me3', 'noiseParameter1_K27me3K36me0', 'noiseParameter1_K27me3K36me1', 'noiseParameter1_K27me3K36me2'

## Create objective function

In [9]:
obj, edatas = importer.create_objective()
print(amici.getDataObservablesAsDataFrame(obj.amici_model, edatas))

print(edatas[0].fixedParametersPreequilibration)
print(obj.dim, obj.x_names, len(obj.x_ids), obj.opt_to_sim_par_mapping)
print(edatas[0].fixedParametersPreequilibration, edatas[0].fixedParameters)
obj.amici_model.setParameterScale(amici.ParameterScaling_log10)
x_nom = manager.parameter_df['nominalValue'].values
print("Nominal parameter values:\n", x_nom)
amici.
obj(x_nom)

   time datatype  t_presim  dilution  dilution_preeq  dilution_presim  \
0   0.0     data       0.0       0.0             1.0              NaN   
1   5.0     data       0.0       0.0             1.0              NaN   
2  10.0     data       0.0       0.0             1.0              NaN   
3  25.0     data       0.0       0.0             1.0              NaN   

   observable_K27me0K36me0  observable_K27me0K36me1  observable_K27me1K36me0  \
0                  0.00431                  0.00789                  0.00216   
1                  0.00093                  0.00499                  0.00159   
2                  0.00051                  0.00308                  0.00123   
3                  0.00054                  0.00131                  0.00091   

   observable_K27me0K36me2             ...               \
0                  0.16777             ...                
1                  0.14469             ...                
2                  0.12140             ...              

55.14462118870135

## Run optimization

In [5]:
optimizer = pypesto.ScipyOptimizer()

problem = importer.create_problem(obj)

# do the optimization
result = pypesto.minimize(problem=problem, optimizer=optimizer,
                          n_starts=10)

AttributeError: 'Importer' object has no attribute 'par_lb'

## Visualize

In [ ]:
import pypesto.visualize

pypesto.visualize.waterfall(result)
pypesto.visualize.parameters(result)

In [ ]:
df = obj.simulations_to_measurement_df(result.optimize_result.get_for_key('x')[0])
plt.xlabel("Experiment")
plt.ylabel("Simulation")
plt.scatter(importer.measurement_df['measurement'], df['measurement'])

In [ ]:
edatas_sim = obj.simulate(x_bst)
df_sim = importer.to_measurement_df(edatas_sim)
plot_simulation(manager.measurement_df, [df_sim])